### Load dataset from Kaggle

In [1]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [2]:
# .kaggle 디렉토리 생성및 kaggle.json 파일 폴더에 복사
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
# change authorization of Kaggle Dataset API
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
%cd /content
!kaggle datasets download -d carlosrunner/pizza-not-pizza

/content
 84% 85.0M/101M [00:00<00:00, 195MB/s]
100% 101M/101M [00:00<00:00, 147MB/s] 


In [5]:
!unzip pizza-not-pizza.zip

Archive:  pizza-not-pizza.zip
  inflating: pizza_not_pizza/food101_subset.py  
  inflating: pizza_not_pizza/not_pizza/1005746.jpg  
  inflating: pizza_not_pizza/not_pizza/1009225.jpg  
  inflating: pizza_not_pizza/not_pizza/1012312.jpg  
  inflating: pizza_not_pizza/not_pizza/1018802.jpg  
  inflating: pizza_not_pizza/not_pizza/1020735.jpg  
  inflating: pizza_not_pizza/not_pizza/1027142.jpg  
  inflating: pizza_not_pizza/not_pizza/102839.jpg  
  inflating: pizza_not_pizza/not_pizza/1043233.jpg  
  inflating: pizza_not_pizza/not_pizza/1045400.jpg  
  inflating: pizza_not_pizza/not_pizza/104773.jpg  
  inflating: pizza_not_pizza/not_pizza/1050956.jpg  
  inflating: pizza_not_pizza/not_pizza/1054362.jpg  
  inflating: pizza_not_pizza/not_pizza/1062097.jpg  
  inflating: pizza_not_pizza/not_pizza/1066151.jpg  
  inflating: pizza_not_pizza/not_pizza/1070772.jpg  
  inflating: pizza_not_pizza/not_pizza/1070867.jpg  
  inflating: pizza_not_pizza/not_pizza/107499.jpg  
  inflating: pizza_not_

### Split Dataset

In [6]:
%pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
%cd /content

/content


In [8]:
import splitfolders
splitfolders.ratio('/content/drive/MyDrive/Colab Notebooks/pizza_not_pizza', output='/content/pizzaDataset',
                   seed=42, ratio=(.8, .1, .1))

Copying files: 1966 files [00:31, 62.80 files/s] 


In [9]:
%cd /content/drive/MyDrive/Colab Notebooks/ResNet

/content/drive/MyDrive/Colab Notebooks/ResNet


### main

In [10]:
from dataset import pizza_data_load
from model import *
from evaluate import evaluate 
from torchvision import transforms as T
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm

In [16]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

# Hyperparameters & Optimizer & Loss
batch_size = 32
train_loader, valid_loader, test_loader =\
    pizza_data_load(path='/content/pizzaDataset/',batch_size=batch_size,transforms=transform)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

def resnet34(device):
    return ResNet(BasicBlock, [3, 4, 6, 3]).to(device)

model = resnet34(device)

lr = 1e-3
EPOCHS = 20
optim = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss().to(device)


# Train
best_acc = 0.0
for epoch in tqdm(range(EPOCHS)):
    print("{}/{} Epochs".format(epoch + 1, EPOCHS))
    print('-' * 20)
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)

        pred = model(x)
        _, preds = torch.max(pred, 1)
        loss = loss_func(pred, y)

        optim.zero_grad()
        loss.backward()
        optim.step()
        
    print("Loss is : ", loss.item())

cuda:0


  0%|          | 0/20 [00:00<?, ?it/s]

1/20 Epochs
--------------------
Loss is :  0.897925853729248
2/20 Epochs
--------------------
Loss is :  0.4192926287651062
3/20 Epochs
--------------------
Loss is :  1.5626369714736938
4/20 Epochs
--------------------
Loss is :  0.16183649003505707
5/20 Epochs
--------------------
Loss is :  0.5569781064987183
6/20 Epochs
--------------------
Loss is :  0.49701541662216187
7/20 Epochs
--------------------
Loss is :  0.2960520386695862
8/20 Epochs
--------------------
Loss is :  0.09973078966140747
9/20 Epochs
--------------------
Loss is :  0.2619841992855072
10/20 Epochs
--------------------
Loss is :  0.5719172954559326
11/20 Epochs
--------------------
Loss is :  0.8006464242935181
12/20 Epochs
--------------------
Loss is :  0.31720253825187683
13/20 Epochs
--------------------
Loss is :  0.8213984966278076
14/20 Epochs
--------------------
Loss is :  0.6800172924995422
15/20 Epochs
--------------------
Loss is :  0.19344456493854523
16/20 Epochs
--------------------
Loss is :  

In [20]:
# Eval for train/val set 

evaluate(model, train_loader, device, loss_func)
evaluate(model, valid_loader, device, loss_func)

Accuracy of Test Data: 70.35623168945312
Accuracy of Test Data: 70.91836547851562


In [19]:
# Evaluate

evaluate(model, test_loader, device, loss_func)

Accuracy of Test Data: 76.7676773071289
